In [1]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

c:\Users\Asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_train = pd.read_csv("./training_data/merged_train_i3.csv", sep=';' )

numeric_column_sums = df_train.select_dtypes(include='number').sum()
print(numeric_column_sums)
print(f"Total data: {df_train.shape[0]}")


anger           2981
anticipation    1710
disgust         3036
fear            1895
joy             2883
love            1220
optimism        2305
pessimism       1379
sadness         2331
surprise         796
trust            791
dtype: int64
Total data: 11672


In [4]:
def adaptive_thresholding(predictions, target_count, initial_threshold=0.99, step=0.001):
    
    total_pseudo_labels = 0
    
    while total_pseudo_labels < target_count:

        pseudo_labels = [ (0 if item < initial_threshold else 1) for item in predictions]
        total_pseudo_labels = np.sum(pseudo_labels)
        
        if total_pseudo_labels >= target_count:
            break

        if initial_threshold < 0.1:
            break

        initial_threshold -= step
    
    return initial_threshold

not_chosen_columns = ['ID', 'Tweet']
label_columns = [col for col in df_train.columns if col not in not_chosen_columns]

increase_needed = [140, 282, 143, 246, 137, 202, 109, 229, 111, 183, 181]
pseudo_thresholds = []

df_goemotion = pd.read_csv("./pseudo_labeled_data/pseudo_labeled_dataset_i2.csv", sep=';')

for i in range( len(label_columns) ):
    
    predic_list = df_goemotion[label_columns[i]].tolist()
    result_threshold = adaptive_thresholding(predic_list, increase_needed[i])
    pseudo_thresholds.append(round(result_threshold, 3))

print(pseudo_thresholds)

[0.837, 0.823, 0.816, 0.828, 0.893, 0.79, 0.763, 0.693, 0.778, 0.807, 0.721]


In [ ]:
df_goemotion_result = df_goemotion.copy()

for i in range( len(label_columns) ):
    label = label_columns[i]
    df_goemotion_result[label] = df_goemotion_result[label].apply(lambda val: 0 if val < pseudo_thresholds[i] else 1 )

df_filtered = df_goemotion_result.loc[(df_goemotion_result[label_columns].sum(axis=1) > 0)]
df_remaining = df_goemotion_result.loc[(df_goemotion_result[label_columns].sum(axis=1) == 0)]
df_combined = pd.concat([df_train, df_filtered], ignore_index=True)

df_remaining.to_csv('goemotion_train_i4.csv', sep=';', encoding='utf_8', index=False)
df_combined.to_csv('merged_train_i4.csv', sep=';', encoding='utf_8', index=False)
